In [19]:
import numpy as np
import pandas as pd
import torch
from importlib import reload
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score
from sklearn.impute import SimpleImputer
import midi.utils; reload(midi.utils)
from midi.utils import benchmark_with_multiple_models

In [5]:
rna_seq = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_rnaseq_genes_145subjects_17072genes.npy")
acgh = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_1mb_aCGH.npy")
clinical = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_clinical_data_for_fit.npy")

stages = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_stages.npy")
stages = np.array(stages).flatten().astype(str)
#stages

y = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_outputs.npy")
mergeable = pd.read_csv("/home/vero/projects/camda/NB/data_cleaned/clinical_data_for_fit.csv", index_col=0)
clinical_HR = pd.read_csv("/home/vero/projects/camda/NB/data_cleaned/info_patients_clinical.csv", 
                          index_col=0)['high_risk2']
clinical_HR.sort_index()
HR = clinical_HR.loc[mergeable.index].values
HR[np.where(HR=='HR')] = '1'
HR = HR.astype(int)

In [11]:
sss = StratifiedShuffleSplit(n_splits=3)
train, test = next(sss.split(rna_seq, y[:,1]))

In [20]:
rna_imp = SimpleImputer(axis=1)
rna_tr = rna_imp.fit_transform(rna_seq[train,:])
acgh_tr = acgh[train,:]
acgh_tr[np.where(np.isnan(acgh_tr))] = 0
cl_tr = clinical[train,:]
cl_tr[np.where(np.isnan(cl_tr))] = 0


In [22]:
np.any(np.isnan(rna_tr)), np.any(np.isnan(acgh_tr)), np.any(np.isnan(cl_tr))

(False, False, False)

In [73]:
y_new = y[:,1]
y_new[np.where(y_new==0)] = -1

# Classification Ensemble elastic net

In [294]:
import midi.utils; reload(midi.utils)
import midi.intermediate.ensemble.classification; reload(midi.intermediate.ensemble.classification)

from midi.intermediate.ensemble.classification import EnsembleClassifier

In [342]:
model = EnsembleClassifier(estimator_params=[[0.1],[0.9]], verbose=1, tol=1e-5)

In [343]:
model.fit([rna_tr, acgh_tr, cl_tr], y_new[train])

Iter: 0,	 obj:519.34,	 obj_diff:inf,	 iter_diff: 0.00000
Iter: 1,	 obj:519.13,	 obj_diff:0.20361,	 iter_diff: 0.00000
Iter: 2,	 obj:518.94,	 obj_diff:0.19092,	 iter_diff: 0.00000
Iter: 3,	 obj:518.78,	 obj_diff:0.16442,	 iter_diff: 0.00000
Iter: 4,	 obj:518.63,	 obj_diff:0.15292,	 iter_diff: 0.00000
Iter: 5,	 obj:518.48,	 obj_diff:0.14215,	 iter_diff: 0.00000
Iter: 6,	 obj:518.35,	 obj_diff:0.13243,	 iter_diff: 0.00000
Iter: 7,	 obj:518.23,	 obj_diff:0.12323,	 iter_diff: 0.00000
Iter: 8,	 obj:518.11,	 obj_diff:0.11494,	 iter_diff: 0.00000
Iter: 9,	 obj:518.01,	 obj_diff:0.10709,	 iter_diff: 0.00000
Iter: 10,	 obj:517.91,	 obj_diff:0.10001,	 iter_diff: 0.00000
Iter: 11,	 obj:517.81,	 obj_diff:0.09896,	 iter_diff: 0.00000
Iter: 12,	 obj:517.72,	 obj_diff:0.08689,	 iter_diff: 0.00000
Iter: 13,	 obj:517.64,	 obj_diff:0.08252,	 iter_diff: 0.00000
Iter: 14,	 obj:517.56,	 obj_diff:0.08208,	 iter_diff: 0.00000
Iter: 15,	 obj:517.49,	 obj_diff:0.07081,	 iter_diff: 0.00000
Iter: 16,	 obj:517.42,

Iter: 134,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 135,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 136,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 137,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 138,	 obj:516.34,	 obj_diff:0.00015,	 iter_diff: 0.00000
Iter: 139,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 140,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 141,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 142,	 obj:516.34,	 obj_diff:0.00013,	 iter_diff: 0.00000
Iter: 143,	 obj:516.34,	 obj_diff:0.00011,	 iter_diff: 0.00000
Iter: 144,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 145,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 146,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 147,	 obj:516.34,	 obj_diff:0.00016,	 iter_diff: 0.00000
Iter: 148,	 obj:516.34,	 obj_diff:0.00008,	 iter_diff: 0.00000
Iter: 149,	 obj:516.34,	 obj_diff:0.00011,	 iter_diff: 

In [344]:
model.coef_, model.W_

(array([-0.78606569, -1.10204121,  0.28538946]),
 [array([-1.21389839e-05, -1.80483713e-06, -6.03308958e-07, ...,
          6.68827240e-06,  1.14697324e-05,  1.28220331e-05]),
  array([-3.59537590e-06,  1.23894483e-05,  1.35321896e-06, ...,
          1.02438520e-05,  9.89259959e-07, -3.73495260e-07]),
  array([1.05661530e-04, 7.49649836e-07])])

In [345]:
R = []
for w, X in zip(model.W_,[rna_tr, acgh_tr, cl_tr] ):
    R.append(np.sign(X.dot(w)))
R = np.array(R)
print(R)
print(np.sign(model.coef_.dot(R)))
#model.W_

[[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  -1. -1. -1. -1.]
 [-1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1. -1. -1. -1.  1.  1.  1. -1.
  -1.  1.  1. -1. -1. -1.  1.  1.  1. -1.  1. -1.  1. -1.  1.  1.  1. -1.
  -1. -1.  1. -1. -1.  1.  1.  1.  1. -1. -1.  1. -1.  1.  1.  1.  1. -1.
   1. -1.  1.  1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1. -1. -1.  1.  1.
   1.  1.  1.  1. -1.  1. -1. -1. -1.  1. -1.  1.  1.  1.  1. -1. -1.  1.
  -1.  1.  1. -1.  1.  1.  1. -1.  1. -1.  1.  1.  1.  1. -1. -1.  1. -1.
   1.  1. -1. -1. -

In [346]:
model.predict([rna_tr, acgh_tr, cl_tr])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [347]:
y_new[train]

array([-1,  1, -1, -1, -1,  1, -1,  1, -1,  1, -1, -1,  1, -1, -1, -1, -1,
        1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1,  1, -1,
        1, -1, -1, -1,  1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1,
        1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
        1, -1,  1,  1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1,  1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1,  1,  1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [348]:
model.score([rna_tr, acgh_tr, cl_tr], y_new[train])

0.5

In [340]:
rna_ts = rna_imp.fit_transform(rna_seq[test,:])
acgh_ts = acgh[test,:]
acgh_ts[np.where(np.isnan(acgh_ts))] = 0
cl_ts = clinical[test,:]
cl_ts[np.where(np.isnan(cl_ts))] = 0
y_ts = y[test,1]

In [341]:
model.score([rna_ts, acgh_ts, cl_ts], y_ts)

0.5

In [122]:
from sklearn.linear_model import RidgeClassifier
m = RidgeClassifier()
m.fit(rna_tr, y_new[train])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [123]:
m.coef_

array([[-7.98361748e-06,  9.85689640e-05,  4.60147559e-04, ...,
         2.58399757e-04,  1.73245190e-04, -1.98226621e-04]])